In [1]:
!PYTORCH_JIT_LOG_LEVEL="fusion_pass.cpp" python s.py

/home/taosy/repo/pytorch/vision/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
[W OperatorEntry.cpp:153] Warning: Warning only once for all operators,  other operators may also be overrided.
  Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::conv1d(Tensor input, Tensor weight, Tensor? bias=None, int[1] stride=[1], SymInt[1] padding=[0], int[1] dilation=[1], int groups=1) -> Tensor
    registered at /opt/conda/conda-bld/pytorch_1686640406433/work/build/aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: AutocastCPU
  previous kernel: registered at /opt/conda/conda-bld/pytorch_1686640406433/work/aten/src/ATen/autocast_mode.cpp:216
       new

# fusion_pass provided by IPEX
```
// {IPEX} / csrc / cpu / jit / fusion_pass.cpp
void FusionPass(std::shared_ptr<Graph>& graph) {
  GRAPH_DUMP(
      "Before RemoveProfileNodesAndSpecializeTypes. Beginning of "
      "optimization pass",
      graph);
  RemoveProfileNodesAndSpecializeTypes(graph);

  // LLGA fusion pass for int8
  GRAPH_DUMP(
      "After RemoveProfileNodesAndSpecializeTypes. Before LLGA fusion pass",
      graph);

  if (isQuantized(graph) || fuser::onednn::is_llga_fp32_bf16_enabled()) {
    RemoveRedundantAliases(graph);
    QPaddingConversion(graph);
    fuser::onednn::fuseGraph(graph);
  }
  GRAPH_DUMP(
      "After LLGA fusion pass. Before ReplaceInplaceOpsWitOutplaceOps", graph);

  // Replace inplace ops with outplace ops so that we do not need to enumerate
  // all inplace and outplace fusion patterns. This replacement is the inverse
  // of ApplyInplaceOptimization. All the replaced ops failed to be fused will
  // be reverted by ApplyInplaceOptimization.
  ReplaceInplaceOpsWitOutplaceOps(graph);
  GRAPH_DUMP(
      "After ReplaceInplaceOpsWitOutplaceOps. Before IPEXFusionPass", graph);

  // IPEX fusion pass for fp32 and bf16
  IPEXFusionPass(graph);
  GRAPH_DUMP(
      "After IPEXFusionPass. Before RemoveTensorTypeSpecializations", graph);
  // TODO: workaround here to go throughput the TE fuser pass before
  // RemoveTensorTypeSpecializations since TE fuser needs the type
  // specializations
```

# pass provided by PyTorch
```
(pt-39-4) taosy@eaas-wsf-dev-icx03:~/repo/pytorch/pytorch-nightly-pt39-6$ grep _jit_pass torch/csrc/jit/python/init.cpp 
      .def("_jit_pass_lower_all_tuples", LowerAllTuples)
      .def("_jit_pass_run_decompositions", RunDecompositions)
      .def("_jit_pass_propagate_shapes_on_graph", PropagateShapesOnGraph)
          "_jit_pass_propagate_shapes_on_graph_and_build_compute",
          "_jit_pass_propagate_shapes_on_graph_and_build_compute",
          "_jit_pass_propagate_shapes_on_graph_and_build_compute",
      .def("_jit_pass_integer_value_refinement", RefineIntegerValues)
      .def("_jit_pass_autocast", Autocast)
      .def("_jit_pass_fuse", FuseGraph)
          "_jit_pass_replace_old_ops_with_upgraders",
          "_jit_pass_dce",
          "_jit_pass_dce_allow_deleting_nodes_with_side_effects",
          "_jit_pass_cse",
          "_jit_pass_fuse_quantized_add_relu",
          "_jit_pass_insert_observers",
          "_jit_pass_insert_observer_method_for_ondevice_ptq",
          "_jit_pass_insert_quant_dequant",
          "_jit_pass_insert_quant_dequant_for_ondevice_ptq",
          "_jit_pass_insert_prepack_unpack",
          "_jit_pass_insert_prepack_unpack",
          "_jit_pass_quant_fusion",
          "_jit_pass_fold_convbn",
          ......
```